In [34]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [35]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']], df.bought_insurance,test_size=0.2, random_state=25)

In [37]:
X_train

,age,affordibility
0,22,1
13,29,0
6,55,0
17,58,1
24,50,1
19,18,1
25,54,1
16,25,0
20,21,1
3,52,0


In [38]:
len(X_train)

22

In [40]:
df.shape

(28, 3)

In [41]:
#scaling
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age']/100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age']/100

In [106]:
X_train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [42]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [ ]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)
    
    

Epoch 1/5000
1/1 [==============================] - 0s 299ms/step - loss: 0.7113 - accuracy: 0.5000
Epoch 2/5000
1/1 [==============================] - 0s 15ms/step - loss: 0.7110 - accuracy: 0.5000
Epoch 3/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7106 - accuracy: 0.5000
Epoch 4/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7102 - accuracy: 0.5000
Epoch 5/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7098 - accuracy: 0.5000
Epoch 6/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.7094 - accuracy: 0.5000
Epoch 7/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7091 - accuracy: 0.5000
Epoch 8/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7087 - accuracy: 0.5000
Epoch 9/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7083 - accuracy: 0.5000
Epoch 10/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.7079 - accuracy: 0.5000
Epoch 11/5000
1/

In [50]:
model.evaluate(X_test_scaled, y_test)


1/1 [==============================] - 0s 53ms/step - loss: 0.3550 - accuracy: 1.0000


[0.3549776077270508, 1.0]

In [51]:
X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [52]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 45ms/step


array([[0.7054848 ],
       [0.3556957 ],
       [0.16827832],
       [0.47801185],
       [0.7260696 ],
       [0.8294983 ]], dtype=float32)

In [53]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [ ]:
#The "coef" variable would hold the weights and the "intercept" variable would hold the bias term
#model.get_weights() - used to extract the weights and biases from the model after training.
coef, intercept = model.get_weights()
coef, intercept

In [55]:
#building a neural network from scratch without using tensorflow 
def sigmoid(x):
    import math
    return 1 / (1 +math.exp(-x))
sigmoid(18)



0.9999999847700205

In [56]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

In [21]:
prediction_function(.47, 1)

C:\Users\surja\AppData\Local\Temp\ipykernel_5648\3931472439.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 +math.exp(-x))


0.705484819775958

In [58]:
prediction_function(0.26, 0)

C:\Users\surja\AppData\Local\Temp\ipykernel_5648\3931472439.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 +math.exp(-x))


0.1682783220980067

In [59]:
#implementing gradient descent from scratch
def log_loss(y_true,  y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(y_predicted_new))



In [60]:
#sigmoid function for numpy - insted of takig just 1 no it takes a whole array 
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [66]:
#*** important for interviews definng a gradient descent function- It finds out the weight
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

def fit(self,X, y, epochs, loss_threshold):
    self.w1, self.w2, self.bias = self.gradient_descent(X['age'], X['affordibility'], y, epochs, loss_threshold)
    
def gradient_descent(self, age, affordibility, y_true, epochs, loss_threshold):
    # w1, w1, bias
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    
    for i in range (epochs):
        weighted_sum = w1*age +w2 *affordibility + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)
        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_true))
        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias -rate *bias_d
        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{log_loss}')
        if loss<=loss_threshold:
            break
            
    return w1,w2, bias
        


        
        
        
    




In [67]:
customModel = myNN()
customModel.fit(X_train_scaled, y_train, epochs=5, loss_threshold= 0.5464)


AttributeError: 'myNN' object has no attribute 'fit'

In [29]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000, 0.5464)

Epoch:0, w1:0.974907633470177, w2:0.948348125394529, bias:-0.11341867736368583, loss:<function log_loss at 0x000001BBB44B9800>


(0.974907633470177, 0.948348125394529, -0.11341867736368583)

In [30]:
coef, intercept

(array([[5.060863 ],
        [1.4086521]], dtype=float32),
 array([-2.913703], dtype=float32))

In [ ]:
w1 = 5.060863, w2 = 1.4086521, bias = -2.913703